In [1]:
print('ok')

ok


In [6]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os 

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain 

In [4]:
%pwd

'c:\\Users\\surya\\CampusX\\End-to-end-Source-Code-Analysis\\research'

In [5]:
!mkdir test_repo

In [7]:
repo_path = "test_repo/"

repo = Repo.clone_from('https://github.com/SKC9372/Flask-ml-project',to_path = repo_path)

In [8]:
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes= [".py"],
    parser= LanguageParser(Language.PYTHON,parser_threshold=500)

)

In [9]:
documents = loader.load()

In [10]:
documents

[Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import (\n    Flask,url_for,\n    render_template\n)\nfrom forms import Inputform\nimport joblib\nimport pandas as pd\n\napp = Flask(__name__)\napp.config[\'SECRET_KEY\'] = \'secret_key\'\n\nmodel = joblib.load(\'rf_model.joblib\')\n\n@app.route(\'/\')\n@app.route(\'/home\')\ndef home():\n    return render_template(\'home.html\',title=\'Home\')\n\n@app.route(\'/predict\',methods = [\'GET\',\'POST\'])\ndef predict():\n    form = Inputform()\n    if form.validate_on_submit():\n        X_new = pd.DataFrame(dict(\n             airline=[form.airline.data],\n            date_of_journey=[form.date_of_journey.data.strftime("%Y-%m-%d")],\n            source=[form.source.data],\n            destination=[form.destination.data],\n            dep_time=[form.dep_time.data.strftime("%H:%M:%S")],\n            arrival_time=[form.arrival_time.data.strftime("%H:%M:%S")],\n            dura

In [12]:
len(documents)

2

In [13]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
                                                                  chunk_size = 500,
                                                                  chunk_overlap = 20)

In [14]:
texts = documents_splitter.split_documents(documents)

In [15]:
texts

[Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content="from flask import (\n    Flask,url_for,\n    render_template\n)\nfrom forms import Inputform\nimport joblib\nimport pandas as pd\n\napp = Flask(__name__)\napp.config['SECRET_KEY'] = 'secret_key'\n\nmodel = joblib.load('rf_model.joblib')\n\n@app.route('/')\n@app.route('/home')\ndef home():\n    return render_template('home.html',title='Home')\n\n@app.route('/predict',methods = ['GET','POST'])"),
 Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='def predict():\n    form = Inputform()\n    if form.validate_on_submit():\n        X_new = pd.DataFrame(dict(\n             airline=[form.airline.data],\n            date_of_journey=[form.date_of_journey.data.strftime("%Y-%m-%d")],\n            source=[form.source.data],\n            destination=[form.destination.data],\n            dep_time=[form.dep_time.data.strftime("%H:%M:%S")],\n

In [16]:
from dotenv import load_dotenv
load_dotenv()
GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")

In [17]:
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

In [18]:
embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')


In [21]:
vector_db = Chroma.from_documents(texts,embedding=embeddings,persist_directory= './db')

In [22]:
vector_db.persist()

C:\Users\surya\AppData\Local\Temp\ipykernel_9288\2766552738.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


In [23]:
llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro',max_tokens=500,temperature=0.3)

In [24]:
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)

C:\Users\surya\AppData\Local\Temp\ipykernel_9288\2995208689.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)


In [34]:
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_db.as_retriever(search_type="mmr", search_kwargs={"k": 3}),
    memory=memory  # Attach memory to the chain
)

In [36]:
question = "What is Inputform function?"

# Pass both the question and chat_history to the chain
result = qa({"question": question})

print(result['answer'])


`Inputform` is a class that inherits from `FlaskForm`. It defines the input fields for an airline price prediction form.  It includes fields for:

* **airline:** A dropdown menu (`SelectField`) populated with unique airline names from a dataset called `X_data`.
* **source:** A dropdown menu (`SelectField`) populated with unique source locations from `X_data`.
* **destination:** A dropdown menu (`SelectField`) for selecting the destination.  The code snippet cuts off and doesn't show how the destination choices are populated.



In [37]:
question = "What is predict function? Explain in detail"

result = qa({"question": question})

print(result['answer'])

The provided code snippets show parts of a Flask web application for flight price prediction.  The `predict` function itself is incomplete. However, based on the available code, we can infer its purpose and reconstruct a likely implementation.

**Purpose:**

The `predict` function takes user input from a web form, processes it, uses a pre-trained machine learning model (`rf_model.joblib`) to predict a flight price, and likely returns the prediction to the user.

**Breakdown:**

1. **Input Handling:**
   - `form = Inputform()`: Creates an instance of the `Inputform` class (defined in `forms.py`, but not provided in the context). This form likely contains fields for flight details like airline, date, source, destination, departure and arrival times, and duration.
   - `if form.validate_on_submit():`: Checks if the form has been submitted and if all the required fields are valid.

2. **Data Preprocessing:**
   - `X_new = pd.DataFrame(...)`: Creates a Pandas DataFrame from the user's input